In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from openai import OpenAI
import os
import geopandas as gpd
from langchain_experimental.agents.agent_toolkits import (
    create_pandas_dataframe_agent,
)
from langchain_openai import OpenAI as OpenAI_langchain


client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)


df_community = pd.read_parquet("all_poster_commenter_details.parquet")
df_issues = pd.read_parquet("issue_details_with_posters_small.parquet")

In [4]:
df_issues

body  \
0    I use Linux at work but at home I have windows...   
1    <!--\r\n\r\nThanks for opening an issue! To he...   
2    <!--\r\n\r\nThanks for opening an issue! To he...   
3    **Describe the bug**\r\n\r\nIt takes a few sec...   
4    This corresponds to 'escapechar' parameter in ...   
..                                                 ...   
870  For the overload of replace in libcudf where i...   
871  Reproducible code:\r\n```\r\nusing f_wrapper =...   
872  **Is your feature request related to a problem...   
873  Currently, when converting floating point type...   
874  When converting floating point types to fixed ...   

                                              comments  \
0    [{'id': 'MDEyOklzc3VlQ29tbWVudDMxNDEzOTU1MQ=='...   
1    [{'id': 'MDEyOklzc3VlQ29tbWVudDQzMzI1NzI1NQ=='...   
2    [{'id': 'MDEyOklzc3VlQ29tbWVudDQ0OTE1NTg1Mw=='...   
3    [{'id': 'MDEyOklzc3VlQ29tbWVudDQ2NjU0OTA3Ng=='...   
4    [{'id': 'MDEyOklzc3VlQ29tbWVudDQ1OTUwMzk2NA=='...   
..                                                 ...   
870  [{'id': 'IC_kwDOBWUGps5-9OuK', 'author': {'log...   
871                                                 []   
872                                                 []   
873                                                 []   
874  [{'id': 'IC_kwDOBWUGps5_dJfp', 'author': {'log...   

                                            commenters             createdAt  \
0    ['SteffenRoe', 'marcodelmoral', 'beckernick', ...  2017-07-01T05:54:03Z   
1    ['harrism', 'hcho3', 'beckernick', 'vyasr', 'P...  2018-09-09T17:57:58Z   
2    ['harrism', 'GregoryKimball', 'vyasr', 'teju85...  2018-08-29T18:00:18Z   
3    ['harrism', 'lmeyerov', 'shwina', 'GregoryKimb...  2019-01-04T02:46:53Z   
4                 ['OlivierNV', 'harrism', 'mjsamoht']  2019-01-16T23:00:29Z   
..                                                 ...                   ...   
870                        ['lithomas1', 'davidwendt']  2024-05-24T16:46:09Z   
871                                                 []  2024-05-24T20:58:21Z   
872                                                 []  2024-05-24T23:51:20Z   
873                                                 []  2024-05-29T21:36:01Z   
874      ['pmattione-nvidia', 'davidwendt', 'ttnghia']  2024-05-29T21:44:15Z   

                                                labels  milestone  \
0    [{'id': 'MDU6TGFiZWw1OTk2MjY1NjE=', 'name': 'f...        NaN   
1    [{'id': 'MDU6TGFiZWwxMTM5NzUwMDA1', 'name': 'C...        NaN   
2    [{'id': 'MDU6TGFiZWw1OTk2MjY1NjE=', 'name': 'f...        NaN   
3    [{'id': 'MDU6TGFiZWw1OTk2MjY1NjE=', 'name': 'f...        NaN   
4    [{'id': 'MDU6TGFiZWw1OTk2MjY1NjE=', 'name': 'f...        NaN   
..                                                 ...        ...   
870  [{'id': 'MDU6TGFiZWw1OTk2MjY1NjE=', 'name': 'f...        NaN   
871  [{'id': 'MDU6TGFiZWw1OTk2MjY1NTk=', 'name': 'b...        NaN   
872  [{'id': 'MDU6TGFiZWw1OTk2MjY1NjE=', 'name': 'f...        NaN   
873  [{'id': 'MDU6TGFiZWw1OTk2MjY1NTk=', 'name': 'b...        NaN   
874  [{'id': 'MDU6TGFiZWw1OTk2MjY1NTk=', 'name': 'b...        NaN   

                                          projectCards  \
0    [{'project': {'name': 'Feature Planning'}, 'co...   
1    [{'project': {'name': 'Other Issues'}, 'column...   
2                                                   []   
3    [{'project': {'name': 'Feature Planning'}, 'co...   
4    [{'project': {'name': 'Feature Planning'}, 'co...   
..                                                 ...   
870                                                 []   
871                                                 []   
872                                                 []   
873                                                 []   
874                                                 []   

                                        reactionGroups  \
0    [{'content': 'THUMBS_UP', 'users': {'totalCoun...   
1                                                   [

In [23]:
external_users = df[df["is_nvidia_employee"] == False]
company_counts = external_users['company'].value_counts()

# Plotting the histogram
plt.figure(figsize=(10,6))
company_counts.plot(kind='bar')
plt.title('Histogram of Companies')
plt.xlabel('Company')
plt.ylabel('Count')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

In [24]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "What type of company is Walmart?",
        }
    ],
    model="gpt-3.5-turbo",
)
chat_completion.choices[0].message.content

"Walmart is a multinational retail corporation that operates a chain of hypermarkets, discount department stores, and grocery stores. It is the world's largest company by revenue and one of the largest employers globally."

In [25]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "What type of company is Halliburton?",
        }
    ],
    model="gpt-3.5-turbo",
)
chat_completion.choices[0].message.content

"Halliburton is an American multinational corporation that provides a variety of products and services to the energy industry, primarily oil and gas exploration and production. They are one of the world's largest oilfield services companies."

In [26]:
gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df["location_lon"], df["location_lat"]),
    crs="epsg:4326",
)

In [30]:
gdf[["login", "name", "followers", "company", "is_nvidia_employee", "geometry"]].explore(column="is_nvidia_employee", cmap="viridis")

In [31]:
df = pd.read_parquet("issue_details_with_posters_small.parquet")

In [45]:
df

login          id               node_id  \
0        AdamKorcz  44787359.0  MDQ6VXNlcjQ0Nzg3MzU5   
1       AjayThorve  20476096.0  MDQ6VXNlcjIwNDc2MDk2   
2     BassemKaroui  23449512.0  MDQ6VXNlcjIzNDQ5NTEy   
3         ChrisJar  13131098.0  MDQ6VXNlcjEzMTMxMDk4   
4    FilippoSimini  22484769.0  MDQ6VXNlcjIyNDg0NzY5   
..             ...         ...                   ...   
243     stevemarin  11840348.0  MDQ6VXNlcjExODQwMzQ4   
244         teju85    532947.0  MDQ6VXNlcjUzMjk0Nw==   
245       trxcllnt    178183.0  MDQ6VXNlcjE3ODE4Mw==   
246     yanxiu0614  11772496.0  MDQ6VXNlcjExNzcyNDk2   
247      yutiansut  11537812.0  MDQ6VXNlcjExNTM3ODEy   

                                            avatar_url gravatar_id  \
0    https://avatars.githubusercontent.com/u/447873...               
1    https://avatars.githubusercontent.com/u/204760...               
2    https://avatars.githubusercontent.com/u/234495...               
3    https://avatars.githubusercontent.com/u/131310...               
4    https://avatars.githubusercontent.com/u/224847...               
..                                                 ...         ...   
243  https://avatars.githubusercontent.com/u/118403...               
244  https://avatars.githubusercontent.com/u/532947...               
245  https://avatars.githubusercontent.com/u/178183...               
246  https://avatars.githubusercontent.com/u/117724...               
247  https://avatars.githubusercontent.com/u/115378...               

                                            url  \
0        https://api.github.com/users/AdamKorcz   
1       https://api.github.com/users/AjayThorve   
2     https://api.github.com/users/BassemKaroui   
3         https://api.github.com/users/ChrisJar   
4    https://api.github.com/users/FilippoSimini   
..                                          ...   
243     https://api.github.com/users/stevemarin   
244         https://api.github.com/users/teju85   
245       https://api.github.com/users/trxcllnt   
246     https://api.github.com/users/yanxiu0614   
247      https://api.github.com/users/yutiansut   

                             html_url  \
0        https://github.com/AdamKorcz   
1       https://github.com/AjayThorve   
2     https://github.com/BassemKaroui   
3         https://github.com/ChrisJar   
4    https://github.com/FilippoSimini   
..                                ...   
243     https://github.com/stevemarin   
244         https://github.com/teju85   
245       https://github.com/trxcllnt   
246     https://github.com/yanxiu0614   
247      https://github.com/yutiansut   

                                         followers_url  \
0     https://api.github.com/users/AdamKorcz/followers   
1    https://api.github.com/users/AjayThorve/followers   
2    https://api.github.com/users/BassemKaroui/foll...   
3      https://api.github.com/users/ChrisJar/followers   
4    https://api.github.com/users/FilippoSimini/fol...   
..                                                 ...   
243  https://api.github.com/users/stevemarin/followers   
244      https://api.github.com/users/teju85/followers   
245    https://api.github.com/users/trxcllnt/followers   
246  https://api.github.com/users/yanxiu0614/followers   
247   https://api.github.com/users/yutiansut/followers   

                                         following_url  \
0    https://api.github.com/users/AdamKorcz/followi...   
1    https://api.github.com/users/AjayThorve/follow...   
2    https://api.github.com/users/BassemKaroui/foll...   
3    https://api.github.com/users/ChrisJar/followin...   
4    https://api.github.com/users/FilippoSimini/fol...   
..                                                 ...   
243  https://api.github.com/users/stevemarin/follow...   
244  https://api.github.com/users/teju85/following{...   
245  https://api.github.com/users/trxcllnt/followin...   
246  https://api.github.com/users/yanxiu0614/follow...   
247  https://api.github.com/users/yutiansut/followi...   


In [36]:
df[df["company"] == "Walmart"]["body"].values[0]

"having predicate pushdowns such as bloom filters work as in hive to skip over sections of data on disk during the read would enable better overall performance as the data would never have to be loaded to memory and then discarded if the predicate didn't match in the row group.\r\n\r\nsupport predicate pushdown similar to what is done with hive in read_orc, primary support for bloom filter but also for sorted data as well to enable only loading relevant data to query.\r\n\r\n\r\n"

In [40]:
content = (
    "Can you summarize this GitHub issue in the cudf repo? "
    f"{df[df['company'] == 'Walmart']['body'].values[0]}"
)
content

"Can you summarize this GitHub issue in the cudf repo? having predicate pushdowns such as bloom filters work as in hive to skip over sections of data on disk during the read would enable better overall performance as the data would never have to be loaded to memory and then discarded if the predicate didn't match in the row group.\r\n\r\nsupport predicate pushdown similar to what is done with hive in read_orc, primary support for bloom filter but also for sorted data as well to enable only loading relevant data to query.\r\n\r\n\r\n"

In [41]:
content = (
    "Can you summarize this GitHub issue in the cudf repo? "
    f"{df[df['company'] == 'Walmart']['body'].values[0]}"
)
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": content
        }
    ],
    model="gpt-3.5-turbo",
)
chat_completion.choices[0].message.content

'This GitHub issue in the cudf repo is discussing the implementation of predicate pushdowns, specifically using bloom filters, to improve performance during data reads. By allowing the system to skip over sections of data on disk that do not match the predicate, unnecessary loading of data into memory can be avoided, leading to better overall performance. The primary focus is on supporting bloom filters, but also considering support for sorted data to efficiently load only relevant data for queries.'

In [43]:
agent = create_pandas_dataframe_agent(OpenAI_langchain(temperature=0), df, verbose=True)

In [44]:
agent.invoke("how many rows are there?")["output"]



> Entering new AgentExecutor chain...
Thought: I need to count the number of rows in the dataframe.
Action: python_repl_ast
Action Input: len(df)248248 rows is the number of rows in the dataframe.
Final Answer: There are 248 rows in the dataframe.

> Finished chain.


'There are 248 rows in the dataframe.'